# Impoprt Files and Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!unzip "gdrive/My Drive/Tesi/PD_Detection/dataset.zip" -d .
!mv dataset data

Archive:  gdrive/My Drive/Tesi/PD_Detection/dataset.zip
   creating: ./dataset/
  inflating: ./dataset/codes.npy     
  inflating: ./dataset/labels.npy    
  inflating: ./dataset/out_data.npy  
  inflating: ./dataset/rest_data.npy  
  inflating: ./dataset/rtn_data.npy  


In [ ]:
%cp gdrive/MyDrive/Tesi/PD_Detection/data_manager.py .

# Import Libreries



In [ ]:
import data_manager as dm
import tensorflow as tf
import math
import numpy as np
import random
import cv2

import logging
from tensorflow import keras
from matplotlib import pyplot as plt

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

tf.get_logger().setLevel(logging.ERROR)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Dropout
import tensorflow.keras as keras


# Original Network

In [ ]:
def get_original(input_shape):
    
    model = Sequential()
    model.add(Conv1D(filters=8, kernel_size=5, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=16, kernel_size=5, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=32, kernel_size=4, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=32, kernel_size=4, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=64, kernel_size=4, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=64, kernel_size=4, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=128, kernel_size=4, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
     
    return model

# With softmax

In [ ]:
def get_original_soft(input_shape):
    
    model = Sequential()
    model.add(Conv1D(filters=8, kernel_size=5, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=16, kernel_size=5, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=32, kernel_size=4, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=32, kernel_size=4, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=64, kernel_size=4, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=64, kernel_size=4, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=128, kernel_size=4, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(2, activation='softmax'))
     
    return model

# 1D ResNet

In [ ]:
def conv_block(n_filters):

    conv = Sequential()

    conv.add(keras.layers.Conv1D(filters=n_filters, kernel_size=8, padding='same'))
    conv.add(keras.layers.BatchNormalization())
    conv.add(keras.layers.Activation('relu'))

    conv.add(keras.layers.Conv1D(filters=n_filters, kernel_size=5, padding='same'))
    conv.add(keras.layers.BatchNormalization())
    conv.add(keras.layers.Activation('relu'))

    conv.add(keras.layers.Conv1D(filters=n_filters, kernel_size=3, padding='same'))
    conv.add(keras.layers.BatchNormalization())

    return conv

In [ ]:
def shortcut(n_filters = None):

    shortcut = Sequential()

    # if n_filters is set, the shape of the input is changed according to it
    if n_filters:
      shortcut.add(keras.layers.Conv1D(filters=n_filters, kernel_size=1, padding='same'))

    shortcut.add(keras.layers.BatchNormalization())

    return shortcut

In [ ]:
def get_resnet(input_shape):

    input_layer = keras.layers.Input(input_shape)

    # Block 1
    conv_1 = conv_block(n_filters = 64)(input_layer)
    short_1 = shortcut(n_filters = 64)(input_layer)

    output_1 = keras.layers.add([conv_1, short_1])
    output_1 = keras.layers.Activation('relu')(output_1)

    # Block 2
    conv_2 = conv_block(n_filters = 128)(output_1)
    short_2 = shortcut(n_filters = 128)(output_1)

    output_2 = keras.layers.add([conv_2, short_2])
    output_2 = keras.layers.Activation('relu')(output_2)

    # Block 3
    conv_3 = conv_block(n_filters = 128)(output_2)
    short_3 = shortcut()(output_2)

    output_3 = keras.layers.add([conv_3, short_3])
    output_3 = keras.layers.Activation('relu')(output_3)

    # Final output
    final_output = keras.layers.GlobalAveragePooling1D(name="features")(output_3)
    final_output = keras.layers.Dense(1, activation='sigmoid')(final_output)  
    model = keras.models.Model(inputs=input_layer, outputs=final_output)
    
    return model

# HMM Classifier

In [ ]:
class HMM_classifier():

  def __init__(self, n_outputs, n_signals, n_states, n_gmodels):

    self.n_outputs = n_outputs
    self.n_signals = n_signals
    self.n_states = n_states
    self.n_gmodels = n_gmodels
    self.models = []

  
  def __format_input(self, X):

    if len(X.shape) < 3:
      X = np.expand_dims(X, 2)

    lens = np.repeat(X.shape[1], X.shape[0])

    return np.concatenate(X), lens


  def fit(self, X, y):

    predictions = []

    for label in range(self.n_outputs):

      inds = np.where(y == label)[0]

      for i in range(self.n_signals):
        hmm = GMMHMM(self.n_states, self.n_gmodels)

        data = X[i].copy()
        data, lens = self.__format_input(data[inds])

        hmm.fit(data, lens)

        self.models.append(hmm)


    print("ciao")
    self.models = np.reshape(self.models, (self.n_signals,-1))


  def predict(self, X):

    total_output = []
    
    for i, sig in enumerate(X):
      if len(sig.shape) < 3:
        sig = np.expand_dims(sig, 2)

      for hmm in self.models[i]:
        preds = []
        for seq in sig:
          preds.append(hmm.score(seq))
        total_output.append(np.array(preds))

    total_output = np.array(total_output)

    print(total_output.shape)

    return self.__compute_final_preds(total_output.T)

  def __compute_final_preds(self, hmm_out):
    preds = []
    for row in hmm_out:
      row = np.reshape(row,(self.n_signals,-1))
      pred = np.argmax(np.sum(row, axis=0).flatten())
      preds.append(pred)
    return np.array(preds)



In [ ]:
!pip install hmmlearn

In [ ]:
from hmmlearn.hmm import GMMHMM


# Trasformations

In [ ]:
def norm_axis(a,b,c):
    newa=a/(math.sqrt(float(a*a+b*b+c*c)))
    newb=b/(math.sqrt(float(a*a+b*b+c*c)))
    newc=c/(math.sqrt(float(a*a+b*b+c*c)))
    
    return ([newa,newb,newc])


def rotation_matrix(axis, theta):
    axis = np.asarray(axis)
    axis = axis/math.sqrt(np.dot(axis, axis))
    
    a = math.cos(theta/2.0)
    b, c, d = -axis*math.sin(theta/2.0)
    
    aa, bb, cc, dd = a*a, b*b, c*c, d*d
    bc, ad, ac, ab, bd, cd = b*c, a*d, a*c, a*b, b*d, c*d
    
    return np.array([[aa+bb-cc-dd, 2*(bc+ad), 2*(bd-ac)], 
                     [2*(bc-ad), aa+cc-bb-dd, 2*(cd+ab)], 
                     [2*(bd+ac), 2*(cd-ab), aa+dd-bb-cc]])


def rand_rotate(image): ## theta: angle, a, b, c, eular vector
    theta = random.random()*math.pi*2
    a = random.random()
    b = random.random()
    c = random.random()
    
    axis=norm_axis(a,b,c)
    rot_matrix = rotation_matrix(axis, theta).T
    
    if image.shape[1] == 6:
        
        # Multipy the rotation matrix for both acceleration and rotation signals
        imagenew_acc = np.dot(image[:,:3], rot_matrix)
        imagenew_rot = np.dot(image[:,3:],rot_matrix)
        
        return np.concatenate([imagenew_acc, imagenew_rot], 1)
    
    elif image.shape[1] == 3:
        
        return np.dot(image, rot_matrix)
    
    else:
        
        # Implement for other types of data with different channels
        
        raise Exception("Cahnnels Error: rotateC works only on sequences with 3 or 6 channels")


def rand_rescale_frequency(image,min_scale, max_scale):
    r = random.random()
    scale = r * (max_scale - min_scale) + min_scale
    
    [x,y]= image.shape
    y1=y
    x1=int(x*scale)
    image=cv2.resize(image,(y1,x1))
    new=np.zeros((x,y))
    if (x1>x):
        start=0
        end=start+x
        new=image[start:end]
    else:
        new_start=0
        new_end=new_start+x1
        new[new_start:new_end]=image
    return new

# Excecution

Create logs direcory

In [ ]:
%rm -r logs
%mkdir logs

Parameters

In [ ]:
batch_size = 64
epochs = 20
min_scale = 0.8
max_scale = 1.2

size = 5000
activities = ["out", "rtn", "rest"]

## CNN vs ResNet

In [ ]:
labels = dm.load_labels().astype(int)
codes = dm.load_codes()

le = LabelEncoder()
codes = le.fit_transform(codes)

model1_accs = []
model2_accs = []
for act in activities:
  data = dm.load_data(act, signal = "acc")

  gss = GroupShuffleSplit(n_splits=1, train_size=0.5, random_state=1)
  for train_i, _ in gss.split(data, labels, groups=codes):
    train_i = train_i

  data = data[train_i]
  train_labels = labels[train_i]

  n_samples = data.shape[0]
  seq_len = data.shape[1]
  channels = data.shape[2]

  train_set, val_set, train_labels, val_labels = train_test_split(data, train_labels, test_size=0.33, random_state=1)

  train_set = np.array([rand_rotate(rand_rescale_frequency(x, min_scale, max_scale)) for x in train_set])

  train_set = tf.data.Dataset.from_tensor_slices((train_set, train_labels))
  val_set = tf.data.Dataset.from_tensor_slices((val_set, val_labels))

  train_set = train_set.batch(batch_size)
  val_set = val_set.batch(batch_size)

  # Original CNN
  model1 = get_resnet((seq_len, channels))

  check_point = tf.keras.callbacks.ModelCheckpoint("gdrive/My Drive/Tesi/PD_Detection/resnet/{}_cnn".format(act), 
                                                  monitor='val_loss', 
                                                  save_weights_only=True, 
                                                  save_best_only=True)

  model1.compile(optimizer = tf.keras.optimizers.Adam(),
                loss = tf.keras.losses.BinaryCrossentropy(),
                metrics = ['AUC'])

  history1 = model1.fit(train_set,
                        epochs = epochs, 
                        batch_size = batch_size, 
                        validation_data = val_set,
                        callbacks = [check_point],
                        verbose = 1)
  
  
  model1_accs.append(np.max(history1.history["val_auc"]))

  # ResNet
  model2 = get_resnet((seq_len, channels))

  check_point2 = tf.keras.callbacks.ModelCheckpoint("gdrive/My Drive/Tesi/PD_Detection/resnet/{}_resnet".format(act), 
                                                  monitor='val_loss', 
                                                  save_weights_only=True, 
                                                  save_best_only=True)

  model.compile(optimizer = tf.keras.optimizers.Adam(),
                loss = tf.keras.losses.BinaryCrossentropy(),
                metrics = ['AUC'])

  history2 = model2.fit(train_set,
                        epochs = epochs, 
                        batch_size = batch_size, 
                        validation_data = val_set,
                        callbacks = [check_point],
                        verbose = 1)
  
  
  model1_accs.append(np.max(history1.history["val_auc"]))
  model2_accs.append(np.max(history2.history["val_auc"]))
  

print(activities)
print(model1_accs)
print(model2_accs)

## Improved ResNet Versions

In [ ]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

labels = dm.load_labels().astype(int)
codes = dm.load_codes()

le = LabelEncoder()
codes = le.fit_transform(codes)

total_val_features = []
total_test_features = []
total_test_predictions = []

gss = GroupShuffleSplit(n_splits=1, train_size=0.5, random_state=1)

for act in activities:
  data = dm.load_data(act, signal = "acc")

  seq_len = data.shape[1]
  channels = data.shape[2]

  for train_i, test_i in gss.split(data, labels, groups=codes):
    train_i = train_i
    test_i = test_i

  train_set = data[train_i]
  train_labels = labels[train_i]

  test_set = data[test_i]
  test_labels = labels[test_i]

  train_set, val_set, train_labels, val_labels = train_test_split(train_set, train_labels, test_size=0.33, random_state=1)

  train_set = np.array([rand_rotate(rand_rescale_frequency(x, min_scale, max_scale)) for x in train_set])

  train_set = tf.data.Dataset.from_tensor_slices((train_set, train_labels))
  val_set = tf.data.Dataset.from_tensor_slices((val_set, val_labels))

  train_set = train_set.batch(batch_size)
  val_set = val_set.batch(batch_size)

  model = get_resnet((seq_len, channels))

  model.load_weights("gdrive/My Drive/Tesi/PD_Detection/resnet/{}_resnet".format(act))

  extractor = tf.keras.models.Model(
    inputs=model.inputs,
    outputs=model.layers[-2].output,
  )  

  val_features = extractor.predict(val_set, batch_size=batch_size)
  total_val_features.append(val_features)

  test_features = extractor.predict(test_set, batch_size=batch_size)
  total_test_features.append(test_features)

  test_predictions = model.predict(test_set, batch_size=batch_size)
  total_test_predictions.append(test_predictions)
  
for _, test_i in gss.split(data, labels, groups=codes):
  test_i = test_i

hmm = HMM_classifier(2,len(activities),8,3)
hmm.fit(total_val_features, val_labels)
hmm_predictions = hmm.predict(total_test_features)

total_val_features = np.concatenate(total_val_features, axis=1)
total_test_features = np.concatenate(total_test_features, axis=1)

svm_classifier = svm.SVC()
svm_classifier.fit(total_val_features, val_labels)

rf_classifier = RandomForestClassifier(random_state=1)
rf_classifier.fit(total_val_features, val_labels)

svm_predictions = svm_classifier.predict(total_test_features)
rf_predictions = rf_classifier.predict(total_test_features)


total_test_predictions = np.concatenate(total_test_predictions).T
total_test_predictions = np.digitize(total_test_predictions, [0.5])
res_predictions = np.array([np.bincount(x).argmax() for x in total_test_predictions])

In [ ]:
np.save("gdrive/My Drive/Tesi/PD_Detection/resnet/hmm_preds",hmm_predictions)
np.save("gdrive/My Drive/Tesi/PD_Detection/resnet/svm_preds",svm_predictions)
np.save("gdrive/My Drive/Tesi/PD_Detection/resnet/rf_preds",rf_predictions)
np.save("gdrive/My Drive/Tesi/PD_Detection/resnet/res_preds",res_predictions)

# Plot Results

Print classification report for each model

In [ ]:
labels = dm.load_labels().astype(int)
codes = dm.load_codes()

le = LabelEncoder()
codes = le.fit_transform(codes)

total_val_features = []
total_test_features = []

gss = GroupShuffleSplit(n_splits=1, train_size=0.5, random_state=1)

for _, test_i in gss.split(data, labels, groups=codes):
  test_i = test_i

y = labels[test_i]

hmm_predictions = np.load("gdrive/My Drive/Tesi/PD_Detection/resnet/hmm_preds.npy")
svm_predictions = np.load("gdrive/My Drive/Tesi/PD_Detection/resnet/svm_preds.npy")
rf_predictions = np.load("gdrive/My Drive/Tesi/PD_Detection/resnet/rf_preds.npy")
res_predictions = np.load("gdrive/My Drive/Tesi/PD_Detection/resnet/res_preds.npy")


print("HMM Results\n")
print(classification_report(y, hmm_predictions, digits = 3))
print("\n AUC:", roc_auc_score(y,hmm_predictions))

print("\n SVM Results\n")
print(classification_report(y, svm_predictions, digits = 3))
print("\n AUC:", roc_auc_score(y,svm_predictions))

print("\n RF Results\n")
print(classification_report(y, rf_predictions, digits = 3))
print("\n AUC:", roc_auc_score(y,rf_predictions))

print("\n ResNet Results\n")
print(classification_report(y, res_predictions, digits = 3))
print("\n AUC:", roc_auc_score(y,res_predictions))

# Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs